# Module 02: Diodes in Practice

**Electricity & Semiconductors Course**

---

In the previous notebook we built up the physics of PN junctions from atoms and doping. Now we put real diodes on a breadboard. We will work with three types:

| Diode | Type | Forward Drop | Key Use |
|-------|------|-------------|--------|
| **1N4148** | Silicon signal diode | ~0.7V | Fast switching, signal processing |
| **1N5817** | Schottky diode | ~0.3V | Low-loss rectification, power |
| **LEDs** | Light-emitting diode | 1.8-3.3V | Indicators, lighting |

By the end of this notebook you will:
- Understand how real diodes differ from the ideal model
- Measure forward voltage drops with your Klein MM300
- Build LED circuits with proper current-limiting resistors
- Build and observe a half-wave rectifier on your oscilloscope
- Understand why Schottky diodes exist and when to use them

## Concept — Real vs. Ideal Diodes

The **ideal diode model** says:
- Forward bias: zero voltage drop, infinite current
- Reverse bias: zero current, infinite resistance

Real diodes are different:
- **Forward voltage drop (V_F):** Silicon diodes drop ~0.6-0.7V, Schottky ~0.2-0.4V, LEDs 1.8-3.3V depending on color
- **Forward resistance:** Current does not become infinite — limited by bulk resistance of silicon
- **Reverse leakage (I_R):** Small but nonzero reverse current (nA to uA)
- **Reverse breakdown (V_BR):** At some reverse voltage, the diode conducts heavily (destructive for regular diodes)
- **Switching speed:** Diodes take finite time to transition from conducting to blocking (reverse recovery time)

### Silicon Signal Diode — 1N4148
- **V_F ≈ 0.7V** at 10mA forward current
- Very fast: reverse recovery time ~4ns
- Max reverse voltage: 100V
- Max forward current: 200mA (continuous)
- Used for: signal clipping, switching, small-signal rectification

### Schottky Diode — 1N5817
- **V_F ≈ 0.3V** at typical currents
- Metal-semiconductor junction (no minority carrier storage)
- Extremely fast switching
- Max reverse voltage: 20V (lower than silicon diodes)
- Max forward current: 1A
- Used for: power rectification, protection, OR-ing power supplies

### LEDs
- Forward voltage determined by **bandgap** of semiconductor material
- Red: ~1.8-2.0V (GaAsP), Yellow: ~2.0-2.2V, Green: ~2.0-3.0V, Blue/White: ~3.0-3.3V (GaN)
- **Must** use current-limiting resistor — LEDs will destroy themselves without one
- Typical operating current: 10-20mA for standard indicator LEDs

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

# Compare IV curves: Silicon (1N4148) vs Schottky (1N5817)

k = 1.381e-23
q_e = 1.602e-19
T = 300
Vt = k * T / q_e

V = np.linspace(0, 1.0, 2000)

# 1N4148: Silicon signal diode
Is_si = 2.52e-9     # reverse saturation current
n_si = 1.8           # ideality factor
Rs_si = 0.57         # series resistance (ohms)

# 1N5817: Schottky diode
Is_sch = 31.4e-6     # much higher Is for Schottky
n_sch = 1.05         # closer to ideal
Rs_sch = 0.042       # lower series resistance

# Calculate currents (simplified — ignoring series R for basic curve shape)
I_si = Is_si * (np.exp(np.clip(V / (n_si * Vt), 0, 100)) - 1)
I_sch = Is_sch * (np.exp(np.clip(V / (n_sch * Vt), 0, 100)) - 1)

fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(14, 6))

# Linear plot
ax1.plot(V * 1000, I_si * 1000, 'b-', lw=2.5, label='1N4148 (Silicon)')
ax1.plot(V * 1000, I_sch * 1000, 'r-', lw=2.5, label='1N5817 (Schottky)')
ax1.set_xlabel('Forward Voltage (mV)', fontsize=12)
ax1.set_ylabel('Forward Current (mA)', fontsize=12)
ax1.set_title('Silicon vs Schottky — Linear Scale', fontsize=14, fontweight='bold')
ax1.set_xlim(0, 1000)
ax1.set_ylim(0, 200)
ax1.legend(fontsize=12)
ax1.grid(True, alpha=0.3)

# Annotate forward voltage difference
ax1.axhline(y=10, color='gray', linestyle='--', lw=0.8)
ax1.text(850, 15, '10 mA reference', fontsize=9, color='gray')
ax1.annotate('~0.7V at 10mA', xy=(700, 10), xytext=(750, 60),
             fontsize=10, color='blue',
             arrowprops=dict(arrowstyle='->', color='blue'))
ax1.annotate('~0.3V at 10mA', xy=(320, 10), xytext=(400, 80),
             fontsize=10, color='red',
             arrowprops=dict(arrowstyle='->', color='red'))

# Log plot
V_log = np.linspace(0.01, 0.9, 2000)
I_si_log = Is_si * (np.exp(np.clip(V_log / (n_si * Vt), 0, 100)) - 1)
I_sch_log = Is_sch * (np.exp(np.clip(V_log / (n_sch * Vt), 0, 100)) - 1)

ax2.semilogy(V_log * 1000, I_si_log, 'b-', lw=2.5, label='1N4148 (Silicon)')
ax2.semilogy(V_log * 1000, I_sch_log, 'r-', lw=2.5, label='1N5817 (Schottky)')
ax2.set_xlabel('Forward Voltage (mV)', fontsize=12)
ax2.set_ylabel('Forward Current (A)', fontsize=12)
ax2.set_title('Silicon vs Schottky — Log Scale', fontsize=14, fontweight='bold')
ax2.legend(fontsize=12)
ax2.grid(True, alpha=0.3, which='both')
ax2.set_ylim(1e-9, 10)

plt.tight_layout()
plt.show()

print("Key difference: Schottky turns on ~0.3-0.4V EARLIER than silicon.")
print("This means less power wasted as heat in the diode.")
print(f"Power lost at 1A through 1N4148: ~0.7W")
print(f"Power lost at 1A through 1N5817: ~0.3W")
print(f"Savings: ~60% less heat!")

## The Material Science Why — Schottky vs. Silicon Junctions

**Why is the Schottky forward voltage lower?**

A standard silicon diode has a **P-N junction** — two types of doped silicon meeting. The built-in potential comes from the difference in Fermi levels across the junction (~0.7V for silicon).

A Schottky diode has a **metal-semiconductor junction** — a metal (like platinum or chromium) deposited directly on N-type silicon. The barrier height is determined by the **work function difference** between the metal and semiconductor, which is typically **0.15-0.45V** — significantly lower than a PN junction.

**Why is the Schottky faster?**

In a PN junction, forward bias injects **minority carriers** across the junction (electrons into P, holes into N). When you suddenly reverse-bias the diode, these stored minority carriers must be removed before the depletion region can re-form. This takes time — the **reverse recovery time** (t_rr).

A Schottky diode is a **majority carrier device** — current is carried only by electrons (in the N-type semiconductor). There are **no minority carriers stored**, so switching is nearly instantaneous.

| Property | PN Junction (1N4148) | Schottky (1N5817) |
|----------|---------------------|-------------------|
| Junction type | P-type / N-type | Metal / N-type |
| V_F | ~0.7V | ~0.3V |
| Reverse recovery | ~4 ns | ~10 ns (but no minority storage) |
| Max V_R | 100V | 20V (tradeoff!) |
| Reverse leakage | Low (nA) | Higher (uA) |

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

# LED forward voltage vs color (wavelength)

led_data = {
    'Infrared': {'wavelength': 940, 'vf': 1.2, 'color': '#8b0000'},
    'Red': {'wavelength': 625, 'vf': 1.8, 'color': '#ff0000'},
    'Orange': {'wavelength': 600, 'vf': 2.0, 'color': '#ff8c00'},
    'Yellow': {'wavelength': 585, 'vf': 2.1, 'color': '#ffd700'},
    'Green': {'wavelength': 525, 'vf': 2.2, 'color': '#00cc00'},
    'Blue': {'wavelength': 470, 'vf': 3.0, 'color': '#0066ff'},
    'Violet': {'wavelength': 405, 'vf': 3.2, 'color': '#8b00ff'},
    'UV': {'wavelength': 370, 'vf': 3.4, 'color': '#4b0082'},
}

fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(14, 6))

# Bar chart of forward voltages
names = list(led_data.keys())
vfs = [led_data[n]['vf'] for n in names]
colors = [led_data[n]['color'] for n in names]

bars = ax1.bar(names, vfs, color=colors, edgecolor='black', lw=1.2)
ax1.set_ylabel('Forward Voltage (V)', fontsize=12)
ax1.set_title('LED Forward Voltage by Color', fontsize=14, fontweight='bold')
ax1.set_ylim(0, 4)
ax1.grid(True, alpha=0.3, axis='y')
for bar, vf in zip(bars, vfs):
    ax1.text(bar.get_x() + bar.get_width()/2, bar.get_height() + 0.05,
             f'{vf}V', ha='center', va='bottom', fontsize=10, fontweight='bold')
plt.setp(ax1.xaxis.get_majorticklabels(), rotation=45, ha='right')

# Scatter plot: wavelength vs Vf
wavelengths = [led_data[n]['wavelength'] for n in names]
ax2.scatter(wavelengths, vfs, c=colors, s=200, edgecolors='black', lw=1.5, zorder=5)
for name, wl, vf in zip(names, wavelengths, vfs):
    ax2.annotate(name, (wl, vf), textcoords='offset points',
                 xytext=(0, 12), ha='center', fontsize=9)

# Theoretical line: E = hc/lambda, V = E/q
wl_theory = np.linspace(350, 1000, 200)
h = 6.626e-34
c = 3e8
q_e = 1.602e-19
V_theory = h * c / (wl_theory * 1e-9) / q_e
ax2.plot(wl_theory, V_theory, 'k--', lw=1.5, alpha=0.5, label='E = hc/λ (theoretical)')

ax2.set_xlabel('Wavelength (nm)', fontsize=12)
ax2.set_ylabel('Forward Voltage (V)', fontsize=12)
ax2.set_title('LED Forward Voltage vs Wavelength', fontsize=14, fontweight='bold')
ax2.legend(fontsize=10)
ax2.grid(True, alpha=0.3)
ax2.invert_xaxis()

plt.tight_layout()
plt.show()

print("Key insight: Shorter wavelength (higher energy) photons require HIGHER forward voltage.")
print("This is because V_F ≈ E_photon / q = hc / (λ × q)")
print("The LED must provide at least as much energy as the photon it emits.")

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

# Half-wave rectifier simulation

f = 60               # 60 Hz mains frequency
Vpeak = 5.0          # peak input voltage (from signal generator)
Vf = 0.7             # silicon diode forward drop
t = np.linspace(0, 3/f, 5000)  # 3 full cycles

# Input: sinusoidal AC
Vin = Vpeak * np.sin(2 * np.pi * f * t)

# Output: half-wave rectified
# Diode conducts when Vin > Vf, output = Vin - Vf
# Diode blocks when Vin < Vf, output = 0
Vout_silicon = np.where(Vin > Vf, Vin - Vf, 0)

# Schottky version
Vf_sch = 0.3
Vout_schottky = np.where(Vin > Vf_sch, Vin - Vf_sch, 0)

fig, axes = plt.subplots(3, 1, figsize=(14, 10), sharex=True)

# Input
axes[0].plot(t * 1000, Vin, 'b-', lw=2, label='V_in (AC)')
axes[0].axhline(y=0, color='gray', lw=0.5)
axes[0].set_ylabel('Voltage (V)', fontsize=12)
axes[0].set_title('Half-Wave Rectifier — Input Signal', fontsize=14, fontweight='bold')
axes[0].legend(fontsize=11)
axes[0].grid(True, alpha=0.3)
axes[0].set_ylim(-6, 6)

# Output with silicon diode
axes[1].plot(t * 1000, Vout_silicon, 'r-', lw=2, label=f'V_out (1N4148, V_F={Vf}V)')
axes[1].plot(t * 1000, Vin, 'b-', lw=1, alpha=0.3, label='V_in (reference)')
axes[1].axhline(y=0, color='gray', lw=0.5)
axes[1].set_ylabel('Voltage (V)', fontsize=12)
axes[1].set_title('Half-Wave Rectifier Output — Silicon Diode', fontsize=14, fontweight='bold')
axes[1].legend(fontsize=11)
axes[1].grid(True, alpha=0.3)
axes[1].set_ylim(-1, 6)

# Annotate the lost voltage
axes[1].annotate(f'Lost: {Vf}V\n(diode drop)', xy=(4.2, Vpeak - Vf),
                 xytext=(6, 5.2), fontsize=10,
                 arrowprops=dict(arrowstyle='->', color='red'),
                 color='red', fontweight='bold')

# Output comparison: silicon vs schottky
axes[2].plot(t * 1000, Vout_silicon, 'r-', lw=2, label=f'1N4148 (V_F={Vf}V)')
axes[2].plot(t * 1000, Vout_schottky, 'g-', lw=2, label=f'1N5817 (V_F={Vf_sch}V)')
axes[2].axhline(y=0, color='gray', lw=0.5)
axes[2].set_xlabel('Time (ms)', fontsize=12)
axes[2].set_ylabel('Voltage (V)', fontsize=12)
axes[2].set_title('Comparison: Silicon vs Schottky Rectifier', fontsize=14, fontweight='bold')
axes[2].legend(fontsize=11)
axes[2].grid(True, alpha=0.3)
axes[2].set_ylim(-1, 6)

plt.tight_layout()
plt.show()

# Calculate DC average
V_dc_silicon = np.mean(Vout_silicon)
V_dc_schottky = np.mean(Vout_schottky)
print(f"Half-wave rectified DC average (silicon):  {V_dc_silicon:.2f} V")
print(f"Half-wave rectified DC average (Schottky): {V_dc_schottky:.2f} V")
print(f"Theoretical: Vpeak/π - Vf/2")
print(f"  Silicon:  {Vpeak/np.pi - Vf/2:.2f} V")
print(f"  Schottky: {Vpeak/np.pi - Vf_sch/2:.2f} V")

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

def led_resistor(Vs, Vf, I_led_mA):
    """Calculate current-limiting resistor for an LED.
    
    Args:
        Vs: Supply voltage (V)
        Vf: LED forward voltage (V)
        I_led_mA: Desired LED current (mA)
    
    Returns:
        Resistance in ohms, power dissipated in resistor (mW)
    """
    I_led = I_led_mA / 1000
    R = (Vs - Vf) / I_led
    P_resistor = I_led**2 * R * 1000  # mW
    P_led = Vf * I_led * 1000          # mW
    return R, P_resistor, P_led

# Table: LED resistor calculations for common scenarios
print("=" * 75)
print("LED Current-Limiting Resistor Calculator")
print("=" * 75)
print(f"{'Supply':>8}  {'LED Color':>10}  {'Vf':>5}  {'I_LED':>6}  {'R':>8}  {'R(std)':>8}  {'P_R':>7}  {'P_LED':>7}")
print("-" * 75)

# Standard resistor values (E12 series)
e12 = [100, 120, 150, 180, 220, 270, 330, 390, 470, 560, 680, 820, 1000,
       1200, 1500, 1800, 2200, 2700, 3300, 3900, 4700]

def nearest_e12(R):
    return min(e12, key=lambda x: abs(x - R) if x >= R * 0.9 else float('inf'))

scenarios = [
    (3.3, 'Red', 1.8, 10),
    (3.3, 'Green', 2.2, 10),
    (3.3, 'Blue', 3.0, 10),
    (5.0, 'Red', 1.8, 20),
    (5.0, 'Green', 2.2, 20),
    (5.0, 'Blue', 3.0, 20),
    (12.0, 'Red', 1.8, 20),
    (12.0, 'White', 3.2, 20),
]

for Vs, color, Vf, I_mA in scenarios:
    R, P_R, P_LED = led_resistor(Vs, Vf, I_mA)
    R_std = nearest_e12(R)
    print(f"{Vs:>7.1f}V  {color:>10}  {Vf:>4.1f}V  {I_mA:>4}mA  {R:>7.0f}Ω  {R_std:>7}Ω  {P_R:>5.1f}mW  {P_LED:>5.1f}mW")

print("\n" + "=" * 75)
print("Formula: R = (Vs - Vf) / I_LED")
print("Always round UP to the next standard resistor value (less current = safer).")
print("Rule of thumb: 330Ω works for most LEDs from 5V at ~10mA.")

## Experiment 1 — Measuring Diode Forward Voltage

**Equipment:** Klein MM300 multimeter, 1N4148, 1N5817, assorted LEDs

### Method A: Using the Diode Test Mode

Your Klein MM300 has a dedicated diode test mode (the diode symbol on the selector).

1. Turn the dial to the **diode/continuity** position
2. Connect **red probe to anode** (the end WITHOUT the band), **black probe to cathode** (the end WITH the band)
3. Read the display — it shows the forward voltage in volts

```
    Klein MM300
    ┌──────────┐
    │  0.652 V  │  <-- Forward voltage reading
    │   DIODE   │
    └──┬───┬────┘
       │   │
    RED│   │BLACK
       │   │
       ▼   ▼
    Anode  Cathode
    (no    (band
    band)  side)
    
    1N4148:  ──|◄──
              A  K
              ↑  ↑
            RED  BLACK
```

### Expected Measurements

| Diode | Expected V_F | Your Measurement |
|-------|-------------|------------------|
| 1N4148 | 0.60 - 0.72V | _________ |
| 1N5817 | 0.15 - 0.35V | _________ |
| Red LED | 1.7 - 2.0V | _________ |
| Green LED | 1.9 - 2.4V | _________ |
| Blue LED | 2.8 - 3.3V | _________ |

**Reverse the probes** — you should read "OL" (open loop / overrange), confirming the diode blocks reverse current.

> **Note:** The diode test mode pushes a small test current (~1mA). Your V_F reading may be slightly lower than the rated value, which is typically specified at 10-20mA.

## Experiment 2 — LED Circuit with Current-Limiting Resistor

**Equipment:** Bench power supply, breadboard, 330Ω resistor, red LED, Klein MM300

### Circuit

```
    Bench PSU                    Breadboard
    ┌─────────┐
    │  5.00V   │
    │  0.010A  │     ┌─────[ 330Ω ]─────┬─────|◄|─────┐
    │         +├─────┤                   │    LED       │
    │          │     │                  V_R             │
    │         -├─────┘                              GND─┘
    └─────────┘
    
    V_supply = 5V
    R = 330Ω
    LED V_F ≈ 1.8V (red)
    
    Expected:
    I = (5.0 - 1.8) / 330 = 9.7 mA
    V across resistor: 5.0 - 1.8 = 3.2V
    Power in resistor: 9.7mA × 3.2V = 31 mW
    Power in LED: 9.7mA × 1.8V = 17 mW
```

### Steps

1. Set bench supply to **5.0V**, current limit to **50mA**
2. Build the circuit on breadboard: PSU+ → 330Ω resistor → LED anode → LED cathode → PSU-
3. The LED should light up
4. **Measure voltage across the LED** with your multimeter (DC voltage mode)
5. **Measure voltage across the resistor** with your multimeter
6. Verify: V_LED + V_R = V_supply (Kirchhoff's Voltage Law)
7. Calculate actual current: I = V_R / 330

### Record Your Results

| Measurement | Expected | Actual |
|------------|----------|--------|
| V_LED | ~1.8V | _______ |
| V_R | ~3.2V | _______ |
| I (calculated from V_R/R) | ~9.7mA | _______ |
| V_LED + V_R | 5.0V | _______ |

## Experiment 3 — Half-Wave Rectifier

**Equipment:** Fnirsi 2C53T oscilloscope + signal generator, breadboard, 1N4148, 1kΩ resistor

### Circuit

```
    Signal Generator          Breadboard             Oscilloscope
    ┌────────────┐                                   ┌──────────┐
    │ 1kHz Sine  │     ┌────|◄|────┬─────────┐      │  CH1: Vin│
    │ 5Vpp       ├─────┤  1N4148   │         │      │  CH2:Vout│
    │           +├─────┤           [1kΩ]      ├──────┤          │
    │            │     │           │         │      │          │
    │           -├─────┴───────────┴─────GND─┴──────┤  GND     │
    └────────────┘                                   └──────────┘
    
    Oscilloscope connections:
    - CH1 probe: Signal generator output (before diode) → see input waveform
    - CH2 probe: Across the 1kΩ resistor → see rectified output
    - GND clips: both to circuit ground
```

### Steps

1. Set signal generator to **1 kHz sine wave, 5V peak-to-peak**
2. Build circuit: signal gen → 1N4148 anode → 1N4148 cathode → 1kΩ to ground
3. Connect oscilloscope CH1 to input, CH2 to output
4. Set oscilloscope timebase to ~500 μs/div
5. **Observe:** CH2 should show only the positive halves of the sine wave
6. **Measure the peak output voltage** — it should be about 0.7V less than the input peak

### What to Look For

- Output peak should be **Vpeak - 0.7V** (diode drop)
- Negative half-cycles should be **near zero** (diode blocking)
- The cutoff is not perfectly sharp — transition happens around 0.5-0.6V on the input

## Experiment 4 — Comparing 1N4148 vs 1N5817

**Equipment:** Bench power supply, breadboard, 1N4148, 1N5817, 220Ω resistor, Klein MM300

### Circuit (test one diode at a time)

```
    Bench PSU
    ┌─────────┐
    │  5.00V   │     ┌────|◄|────┬────[ 220Ω ]────┐
    │         +├─────┤  DIODE    │                 │
    │          │     │          V_F                │
    │         -├─────┴─────────────────────────GND─┘
    └─────────┘
    
    Measure V_F across the diode for each type.
    At 5V with 220Ω:
    - 1N4148: I ≈ (5-0.7)/220 ≈ 19.5mA, V_F ≈ 0.7V
    - 1N5817: I ≈ (5-0.3)/220 ≈ 21.4mA, V_F ≈ 0.3V
```

### Steps

1. Build circuit with **1N4148** first
2. Set bench supply to 5.0V, current limit 100mA
3. Measure voltage across the diode (V_F) with multimeter on DC voltage mode
4. Record the supply current reading from the bench supply display
5. Replace 1N4148 with **1N5817** (same circuit)
6. Repeat measurements

### Expected Results

| Parameter | 1N4148 | 1N5817 |
|-----------|--------|--------|
| V_F | ~0.65-0.72V | ~0.25-0.35V |
| Current | ~19-20 mA | ~21-22 mA |
| Power in diode | ~14 mW | ~7 mW |

**The Schottky wastes about half the power in the diode.** This matters enormously in power electronics where diodes carry amps, not milliamps.

## Simulation

Explore these circuits interactively in Falstad:

**Half-Wave Rectifier:**  
[https://www.falstad.com/circuit/circuitjs.html?ctz=CQAgjCAMB0l3BWcBOaAOMB2ALGXyEw1sESQFJIQEBTAWjDACgA3EAFhG2xDz2xRcUFOShJB-AbJQoYU6fmrYlYeQDMAhgBsAzr2WjV0sdOlA](https://www.falstad.com/circuit/circuitjs.html?ctz=CQAgjCAMB0l3BWcBOaAOMB2ALGXyEw1sESQFJIQEBTAWjDACgA3EAFhG2xDz2xRcUFOShJB-AbJQoYU6fmrYlYeQDMAhgBsAzr2WjV0sdOlA)

**Full-Wave Bridge Rectifier:**  
[https://www.falstad.com/circuit/circuitjs.html?ctz=CQAgjCAMB0l3BWcA2aAOMB2ALGXyQkIBOSBEkNPAUwFowwAoANxABYQ98CCOuUKKCnJQkfAUNEp0sFGxlyAdgEMANgGd+q0ZulTZfIA](https://www.falstad.com/circuit/circuitjs.html?ctz=CQAgjCAMB0l3BWcA2aAOMB2ALGXyQkIBOSBEkNPAUwFowwAoANxABYQ98CCOuUKKCnJQkfAUNEp0sFGxlyAdgEMANgGd+q0ZulTZfIA)

Try:
- Watch the current flow animation in both circuits
- Add a filter capacitor across the load and see the ripple reduce
- Change the load resistance and observe the effect on ripple

## Datasheet Connection

Everything we measured maps directly to datasheet parameters:

| What You Measured | Datasheet Parameter | 1N4148 Spec |
|-------------------|--------------------|-----------|
| Forward voltage drop | V_F @ I_F = 10mA | 0.72V typical, 1.0V max |
| Reverse blocking | V_R (max reverse voltage) | 100V |
| Current through circuit | I_F (max forward current) | 200mA continuous |
| Oscilloscope switching | t_rr (reverse recovery time) | 4 ns max |

**Important datasheet lesson:** V_F is specified at a particular **test current**. At lower currents, V_F is lower. At higher currents, V_F is higher. Always check the test conditions!

For the **1N5817 Schottky**:

| Parameter | Spec |
|-----------|------|
| V_F @ 1A | 0.45V max |
| V_R max | 20V (much lower than 1N4148!) |
| I_F max | 1A (much higher than 1N4148) |

> **Tradeoff alert:** The 1N5817 has much lower V_R (20V vs 100V). Schottky diodes trade reverse voltage capability for lower forward drop and faster switching.

## Checkpoint Questions

1. **You have a 3.3V supply and a blue LED (V_F = 3.0V). What value resistor do you need for 10mA? What happens if you accidentally use a red LED (V_F = 1.8V) with the same resistor?**

2. **Why do Schottky diodes have higher reverse leakage current than silicon PN diodes?**

3. **Your half-wave rectifier shows a peak output of 2.1V when the input peak is 2.5V. The diode drop appears to be only 0.4V — but you are using a 1N4148 (silicon). Explain why.**

4. **You need to rectify a 15V signal. Can you use a 1N5817? Why or why not?**

5. **An LED datasheet says maximum forward current is 20mA. You want to be safe and limit to 15mA from a 5V supply. The LED is green with V_F = 2.1V. What resistor value do you choose from standard E12 values?**

6. **In the half-wave rectifier, what happens during the negative half-cycle? Is the diode voltage exactly 0V? Why or why not?**

7. **Why does the LED forward voltage correlate with the color of light it emits?**

---

*Next notebook: [03-zener-diodes-and-breakdown.ipynb](03-zener-diodes-and-breakdown.ipynb) — Zener diodes and engineered breakdown*